## cut-rod problem

&nbsp;*Length i* | 1 2 3 4 &nbsp;5 &nbsp;6 &nbsp;&nbsp;7 &nbsp; 8 &nbsp; 9  &nbsp; 10|
<br />
&nbsp;*price &nbsp;p* &nbsp;| 1 5 8 9 10 17 17 20 24 30|

In [31]:
# an inefficient way
def cut_rod(p,n):
    if n == 0:
        return 0
    q=float('-inf')
    for i in range(n):
        q=max(q,p[i]+cut_rod(p,n-i-1))
    return q

In [58]:
p=[1,5,8,9,10,17,17,20,24,30]

for i in range(1,len(p)+1):
    print(f"The max price for length {i}: ", cut_rod(p,i))


The max price for length 1:  1
The max price for length 2:  5
The max price for length 3:  8
The max price for length 4:  10
The max price for length 5:  13
The max price for length 6:  17
The max price for length 7:  18
The max price for length 8:  22
The max price for length 9:  25
The max price for length 10:  30


In [46]:
# using dp top-down with memorization
def memoized_cut_rod(p,n):
    r=[float('-inf')]*n
    return memoized_cut_rod_aux(p,n,r)

def memoized_cut_rod_aux(p,n,r):
    if r[n-1] >= 0:
        return r[n-1]
    if n == 0:
        q=0
    else:
        q=float('-inf')
        for i in range(n):
            q=max(q,p[i]+memoized_cut_rod_aux(p,n-i-1,r))
    r[n-1]=q
    return q

In [47]:
for i in range(1,len(p)+1):
    print(f"The max price for length {i}: ", memoized_cut_rod(p,i))

The max price for length 1:  1
The max price for length 2:  5
The max price for length 3:  8
The max price for length 4:  10
The max price for length 5:  13
The max price for length 6:  17
The max price for length 7:  18
The max price for length 8:  22
The max price for length 9:  25
The max price for length 10:  30


In [144]:
# bottom up cut rod
def bottom_up_cut(p,n):
    r=[0]*n
    for i in range(1,n):
        q=float('-inf')
        for j in range(i+1):
            q=max(q,p[j]+r[i-j-1])
        r[i] = q
    
    return r[n-1]

In [145]:
print(bottom_up_cut(p,9))

25


In [151]:
# also print the length of first piece
def extended_bottom_up(p,n):
    r=[0]*n
    s=[0]*n
    
    for i in range(1,n):
        q=float('-inf')
        for j in range(i+1):
            if q < p[j]+r[i-j-1]:
                q=p[j]+r[i-j-1]
                s[i] = j
        r[i]=q
        
    return r,s

In [152]:
print(extended_bottom_up(p,10))

([0, 5, 8, 10, 13, 17, 18, 22, 25, 30], [0, 1, 2, 1, 1, 5, 0, 1, 2, 9])


## matrix-chain multiplication


In [218]:
def matrix_multiply(A,B):
    Arow=len(A)
    Acolumn=len(A[0])
    Brow=len(B)
    Bcolumn=len(B[0])
    C=[[0 for col in range(Bcolumn)] for row in range(Arow)]
    if Acolumn != Brow:
        raise IndexError("error: incompatible dimensions")
    
    for i in range(Arow):
        for j in range(Bcolumn):
            C[i][j]=0
            for k in range(Acolumn):
                C[i][j]+=A[i][k]*B[k][j]
    return C

In [219]:
A=[[12,7,3],
    [4,5,6],
    [7,8,9]]
B=[[5,8,1,2],
    [6,7,3,0],
    [4,5,9,1]]
matrix_multiply(A,B)

[[114, 160, 60, 27], [74, 97, 73, 14], [119, 157, 112, 23]]

if A is p x q and B is q x r, C is p x r. The time to compute C is p . q . r  <br />
for example, we want to multiply a matrix chain <A1, A2, A3>. Suppose the dimensions of the matrices are 10x100,100x5,and 5x50,respectively.
1. case1: ((A1A2)A3). The time is 10.100.5+10.5.50=7500
2. case2: (A1(A2A3)). The time is 100.5.50+10.100.50=75000
<br />
**case1 is 10 times faster than case2!**

## Matrix-chain multiplication problem
The m-c multiplication problem: given a chain <A1,A2,...,An> , where for i = 1,2,..n,matrix Ai has dimension pi-1 x pi,fully parenthesize the product A1,A2..An in a way that minimizes the number of scalar multiplications (such as 7500 is better than 75000)

## Applying dynamic programming
1. Charecterize the structure of an optimal solution
2. Recursively define the value of an optimal solution
3. Compute the value of an optimal solution
4. Construct an optimal solution from computed information

In [13]:
def newTable(row,col):
    M=[[0 for c in range(col)] for r in range(row)]
    return M
    
def matrix_chain_order(p):
    n=len(p)
    m=newTable(n,n)
    s=newTable(n,n)
   
    for  l in range(2,n):
        for i in range(1,n-l+1):
            j=i+l-1
            m[i][j]=float('inf')
            for k in range(i,j):
                q=m[i][k]+m[k+1][j]+p[i-1]*p[k]*p[j]
                if q < m[i][j]:
                    m[i][j]=q
                    s[i][j]=k
    
    return m,s

In [14]:
p=[30,35,15,5,10,20,25]
print(matrix_chain_order(p))

([[0, 0, 0, 0, 0, 0, 0], [0, 0, 15750, 7875, 9375, 11875, 15125], [0, 0, 0, 2625, 4375, 7125, 10500], [0, 0, 0, 0, 750, 2500, 5375], [0, 0, 0, 0, 0, 1000, 3500], [0, 0, 0, 0, 0, 0, 5000], [0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 3, 3, 3], [0, 0, 0, 2, 3, 3, 3], [0, 0, 0, 0, 3, 3, 3], [0, 0, 0, 0, 0, 4, 5], [0, 0, 0, 0, 0, 0, 5], [0, 0, 0, 0, 0, 0, 0]])


In [19]:
def print_optimal_solution(s,i,j):
    if i == j:
        print("A"+str(i),end=" ")
    else:
        print("(",end=" ")
        print_optimal_solution(s,i,s[i][j])
        print_optimal_solution(s,s[i][j]+1,j)
        print(")",end=" ")
def main():
    #  A1=30x35, A2=35x15,A3=15x5,...,A6=20x25
    p=[30,35,15,5,10,20,25]
    n=len(p)
    m,s=matrix_chain_order(p)
    
    print("Number of operation: "+str(m[1][n-1]))
    print_optimal_solution(s,1,n-1)

In [20]:
if __name__ == "__main__":
    main()

Number of operation: 15125
( ( A1 ( A2 A3 ) ) ( ( A4 A5 ) A6 ) ) 

## Longest Common Subsequence(LCS)

if X=(A,B,C,B,D,A,B), Y=(B,D,C,A,B,A). the sequence(B,C,A) is a common sequence for X,Y,but not the longest. (B,C,B,A) is LCS

In [43]:
def LCS_length(x,y):
    m=len(x)
    n=len(y)
    
    b=[[0 for _ in range(n+1)]for _ in range(m+1)]
    c=[[0 for _ in range(n+1)]for _ in range(m+1)]
    
    for i in range(m):
        for j in range(n):
            if x[i]== y[j]:
                c[i+1][j+1]=c[i][j]+1
                b[i+1][j+1]='UP_LEFT'
            elif c[i][j+1] >= c[i+1][j]:
                c[i+1][j+1]=c[i][j+1]
                b[i+1][j+1]='UP'
            else:
                c[i+1][j+1]=c[i+1][j]
                b[i+1][j+1]='LEFT'
    return c,b
def print_LCS(b,x,i,j):
    
    if i==0 or j==0:
        return ''
    if b[i][j] == 'UP_LEFT':
        return str(print_LCS(b,x,i-1,j-1))+str(x[i-1])
        
    elif b[i][j] == 'UP':
        return str(print_LCS(b,x,i-1,j))
    else:
        return str(print_LCS(b,x,i,j-1))
    

In [46]:
if __name__ == "__main__":
    x='ABCBDAB'
    y='BDCABA'
    c,b=LCS_length(x,y)
    print("The LCS is ",print_LCS(b,x,len(x),len(y)),", with length ",len(print_LCS(b,x,len(x),len(y))))

The LCS is  BCBA , with length  4


In [90]:
# simplified version
def LCS(x,y):
    m=len(x)
    n=len(y)
    
    dp=[[0 for _ in range(n+1)]for _ in range(m+1)]
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            if x[i-1]==y[j-1]:
                dp[i][j]=dp[i-1][j-1]+1
            else:
                dp[i][j]=max(dp[i][j-1],dp[i-1][j])
    return dp

def print_LCS(dp,x,y,i,j):
    if dp[i][j]==0:
        return ''
    if x[i-1] == y[j-1]:
        return str(print_LCS(dp,x,y,i-1,j-1))+str(x[i-1])
    elif c[i-1][j] > c[i][j-1]:
        return str(print_LCS(dp,x,y,i-1,j))
    else:
        return str(print_LCS(dp,x,y,i,j-1))
                   

In [91]:
if __name__ == "__main__":
    x='ABCBDAB'
    y='BDCABA'
    dp=LCS(x,y)
    print("The LCS is ",print_LCS(dp,x,y,len(x),len(y)),", with length ",len(print_LCS(dp,x,y,len(x),len(y))))

The LCS is  BDAB , with length  4
